In [ ]:
from ase.io import read
from mace.calculators import MACECalculator
# from mace.calculators import mace_mp
import numpy as np

# --- 1. Load CoOx structure (replace with your file or construction)
atoms = read("CoO_1533087.cif", format='cif')  

# --- 2. Attach MACE calculator
calc = MACECalculator(model_paths='./mace-omat-0-medium.model', default_dtype='float64')
# calc = mace_mp(model='medium', default_dtype='float64')
atoms.calc = calc 

In [ ]:
from phonopy.structure.atoms import PhonopyAtoms

# --- 3. Convert ASE Atoms to PhonopyAtoms
def ase_to_phonopy_atoms(ase_atoms):
    return PhonopyAtoms(symbols=ase_atoms.get_chemical_symbols(),
                        positions=ase_atoms.get_positions(),
                        cell=np.array(atoms.cell),
                        masses=ase_atoms.get_masses())

unitcell = ase_to_phonopy_atoms(atoms)

In [ ]:
from ase.visualize import view 

view(atoms, viewer='x3d')

In [ ]:
from phonopy import Phonopy

# --- 4. Define supercell and create Phonopy object
supercell_matrix = [[2, 0, 0],
                    [0, 2, 0],
                    [0, 0, 2]]  # Can be larger if needed

phonon = Phonopy(unitcell, supercell_matrix)
phonon.generate_displacements(distance=0.01)

In [ ]:
# --- 5. Get list of displaced supercells
supercells = phonon.supercells_with_displacements 
len(supercells)

In [ ]:
from ase import Atoms 
# --- 6. Evaluate forces for each displaced structure
forces = []
for i, sc in enumerate(supercells):
    # Convert PhonopyAtoms to ASE Atoms
    ase_supercell = Atoms(
        symbols=sc.symbols,
        positions=sc.positions,
        cell=sc.cell,
        pbc=True,
    )
    ase_supercell.calc = atoms.calc
    f = ase_supercell.get_forces()
    forces.append(f)

# --- 7. Set forces and calculate force constants
phonon.forces = forces
phonon.produce_force_constants()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot()

for i in range(1,6):
    # Optional: Plot phonon DOS
    phonon.run_mesh([10*i, 10*i, 10*i])  # Adjust mesh size as needed
    phonon.run_total_dos()
    dos = phonon.get_total_dos_dict()
    frequencies_THz = dos["frequency_points"]
    frequencies_recipcm = frequencies_THz * 1e12 / (3e10)  # Convert to reciprocal cm
    total_dos = dos["total_dos"]
    ax.plot(frequencies_THz, total_dos, label=f"Mesh {i}", alpha=i*10/60)

ax.set_xlim([0, 18])
ax.legend()
# ax.set_xlabel(r"Frequency (cm$^{-1}$)")	
ax.set_xlabel(r"Frequency (THz)")
ax.set_ylabel("Phonon DOS")
fig.tight_layout()
plt.show()

In [ ]:
# Generate band structure path (Gamma to X to ...)
from phonopy.phonon.band_structure import get_band_qpoints_and_path_connections

# Γ	0.0000000000	0.0000000000	0.0000000000
# K	0.3750000000	0.3750000000	0.7500000000
# L	0.5000000000	0.5000000000	0.5000000000
# U	0.6250000000	0.2500000000	0.6250000000
# W	0.5000000000	0.2500000000	0.7500000000
# W2	0.7500000000	0.2500000000	0.5000000000
# X	0.5000000000	0.0000000000	0.5000000000

# Γ—X—U|K—Γ—L—W—X

path = [
    [[0, 0, 0], [0.5, 0, 0.5], [0.625, 0.25, 0.625]],
    [[0.375, 0.375, 0.75], [0, 0, 0], [0.5, 0.5, 0.5], [0.5, 0.25, 0.75], [0.5, 0, 0.5]]
]
labels = ["Γ", "X", "U", "K", "Γ", "L", "W", "X"]

qpoints, connections = get_band_qpoints_and_path_connections(path, npoints=51)

phonon.run_band_structure(qpoints, path_connections=connections, labels=labels)
phonon.plot_band_structure().show()
plt.ylim([0, 18.5])